# Phishing Website Detection by Machine Learning - Feature Extraction

This notebook aims to collect the relevant data and extract the selective features.

In [104]:
import os
import requests
import pandas as pd
from urllib.parse import urlparse,urlencode
import ipaddress
import re

# 1. Data Collection:

For this project we are required to collect legitimate(0) and phishing URLs.

- Phishing URLs are collected using opensource service called PhishTank, http://data.phishtank.com/data/online-valid.csv.
- Legitimate URLs are collected using dataset provided by University of New Brunswick, https://www.unb.ca/cic/datasets/url-2016.html. The number of legitimate URLs in this collection are 35,300. Among all the files, 'Benign_list_big_final.csv' is the file of our interest. 

## 1.1. Phishing URLs:

Phishing URLs are collected using PhishTank and loaded into the DataFrame.

In [84]:
#Downloading the phishing URLs file

CSV_URL = 'http://data.phishtank.com/data/online-valid.csv'

with open("Dataset/" + os.path.split(CSV_URL)[1] , 'wb') as f, \
        requests.get(CSV_URL, stream=True) as r:
    for line in r.iter_lines():
        f.write(line+'\n'.encode())

In [85]:
path = "Dataset//online-valid.csv"
df_phishing = pd.read_csv(path)

In [86]:
df_phishing.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7373519,http://uat.bakebe.com/de/bw-bank/,http://www.phishtank.com/phish_detail.php?phis...,2021-12-04T19:45:21+00:00,yes,2021-12-04T20:05:02+00:00,yes,Other
1,7373479,https://mercarir.jp.dbctscc.cn,http://www.phishtank.com/phish_detail.php?phis...,2021-12-04T18:04:30+00:00,yes,2021-12-04T19:52:04+00:00,yes,Other
2,7373476,https://www.bancowlnterbank.com/,http://www.phishtank.com/phish_detail.php?phis...,2021-12-04T17:53:44+00:00,yes,2021-12-04T18:10:52+00:00,yes,Other
3,7373471,https://clientedescontaoitau.000webhostapp.com/,http://www.phishtank.com/phish_detail.php?phis...,2021-12-04T17:35:13+00:00,yes,2021-12-04T19:42:55+00:00,yes,Itau
4,7373461,https://verifiedsbcglobal.weebly.com/,http://www.phishtank.com/phish_detail.php?phis...,2021-12-04T16:36:02+00:00,yes,2021-12-04T18:10:52+00:00,yes,Other


In [87]:
df_phishing.isnull().sum()

phish_id             0
url                  0
phish_detail_url     0
submission_time      0
verified             0
verification_time    0
online               0
target               0
dtype: int64

In [88]:
df_phishing.shape

(12479, 8)

We would pick up 5000 samples from the above dataframe randomly.

In [89]:
#Collecting 5,000 Phishing URLs randomly

df_phishing_final = df_phishing.sample(n = 5000, random_state = 12).copy()
df_phishing_final = df_phishing_final.reset_index(drop=True)
df_phishing_final.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7368857,https://luftcomsa.com/wp-includes/fonts/tesco....,http://www.phishtank.com/phish_detail.php?phis...,2021-11-30T13:45:52+00:00,yes,2021-11-30T18:42:36+00:00,yes,Tesco
1,6924120,http://hunterpowersport.com/wp-includes/custom...,http://www.phishtank.com/phish_detail.php?phis...,2021-01-16T14:07:43+00:00,yes,2021-01-16T14:23:19+00:00,yes,Other
2,7046965,https://urlth.me/jvr9U,http://www.phishtank.com/phish_detail.php?phis...,2021-03-27T12:42:21+00:00,yes,2021-03-27T12:44:06+00:00,yes,Other
3,7304838,https://sites.google.com/view/xopauyr/home,http://www.phishtank.com/phish_detail.php?phis...,2021-09-28T08:20:38+00:00,yes,2021-09-28T08:25:37+00:00,yes,Other
4,6376701,http://poligrafiapias.com/Secured-adobe/08fd9f...,http://www.phishtank.com/phish_detail.php?phis...,2020-01-30T02:02:10+00:00,yes,2020-02-06T14:06:27+00:00,yes,Other


In [90]:
df_phishing_final.shape

(5000, 8)

## 1.2. Legitimate URLs:

In [100]:
path = "Dataset//Benign_list_big_final.csv"
df_legitimate = pd.read_csv(path)
df_legitimate.columns = ["URLs"]

In [101]:
df_legitimate.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [102]:
#Collecting 5,000 Legitimate URLs randomly

df_legitimate_final = df_legitimate.sample(n = 5000, random_state = 12).copy()
df_legitimate_final = df_legitimate_final.reset_index(drop=True)
df_legitimate_final.head()

,URLs
0,http://graphicriver.net/search?date=this-month...
1,http://ecnavi.jp/redirect/?url=http://www.cros...
2,https://hubpages.com/signin?explain=follow+Hub...
3,http://extratorrent.cc/torrent/4190536/AOMEI+B...
4,http://icicibank.com/Personal-Banking/offers/o...


In [103]:
df_legitimate_final.shape

(5000, 1)

# 2. Feature Extraction:

In this step, features are extracted from the Legitimate URLs dataset.

The extracted features are categorized into;

1. Address Bar based Features
2. Domain based Features
3. HTML & Javascript based Features

### 2.1. Address Bar based Features:

Following features are considered as address basr based features;

- Domain of URL
- IP Address in URL
- "@" Symbol in URL
- Length of URL
- Depth of URL
- Redirection "//" in URL
- "http/https" in Domain name
- Using URL Shortening Services “TinyURL”
- Prefix or Suffix "-" in Domain

### 2.1.1. Domain of the URL:

In [122]:
# Extracting the domain of the URL

def getDomain(url):
    domain = urlparse(url).netloc
    if re.match(r"^www.", domain):
        domain = domain.replace("www.", "")
    return domain

### 2.1.2. IP Address in the URL:

We would check for the presence of IP address in the URL, because URLs may have IP address instead of domain name. If an IP address is used as an alternative of the domain name in the URL this may be one of the feature of phishig website's URL.

Hence, If the domain part of the URL has IP address, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [128]:
# Returns bool(True/False), whether the URL contains IP Address or not

def checkIP(url):
    try:
        ipaddress.ip_address(url)
        return True
    except:
        return False

### 2.1.3. "@" Symbol in the URL:

We would check for the presence of "@" symbol in the URL, because URLs containing "@" symbol leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol.

Hence, If the "@" symbol is present in the URL, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [133]:
def checkAtSymbol(url):
    if ("@" in url):
        return True
    else:
        return False

### 2.1.4. Length of the URL:

Here we would compute the length of the URL. Phishers can use long URL to hide the doubtful part in the address bar. Therefore, if the length of the URL is greater than or equal 54 characters then the URL is classified as phishing otherwise legitimate.

Hence, If the length of URL >= 54, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [136]:
def lengthUrl(url):
    if(len(url) >= 54):
        return 1
    else:
        return 0

### 2.1.5. Depth of the URL:

Here, we would compute the depth of the URL. This feature calculates the number of sub pages in the given url based on the '/'.

In [150]:
def depthUrl(url):
    depth = 0
    urlSplit = urlparse(url).path.split('/')
    for i in range(len(urlSplit)):
        if(len(urlSplit[i]) != 0):
            depth += 1
    return depth

### 2.1.6. Redirection "//" in the URL:

We would check the presence of "//" in the URL. The existence of “//” within the URL path means that the user will be redirected to another website. We find that if the URL starts with “HTTP”, that means the “//” should appear in the 5th position. However, if the URL employs “HTTPS” then the “//” should appear in 6th position.

If the "//" is anywhere else in the URL, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [159]:
def checkRedirUrl(url):
    pos = url.rfind("//")
    if(pos > 6):
        return 1
    else:
        return 0

### 2.1.7. "http/https" in the Domain name:

We would check for the presence of "http/https" in the domain part of the URL. The phishers may add the “HTTPS” token to the domain part of a URL in order to trick users.

Hence, If the URL has "http/https" in the domain part, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [164]:
def checkHttpDomain(url):
    domain = urlparse(url).netloc
    if("http" in domain or "https" in domain):
        return 1
    else: 
        return 0

### 2.1.8. Using URL Shortening Services “TinyURL”:

URL shortening is a method on the “World Wide Web” in which a URL may be made smaller in length and still lead to the required webpage. This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL.

Hence, If the URL is using Shortening Services, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [170]:
#listing atll the shortening services
all_shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [173]:
# Checking for Shortening Services present in the URL.

def checkShortUrl(url):
    if re.search(all_shortening_services, url):
        return 1
    else:
        return 0

### 2.1.9. Prefix or Suffix "-" in the Domain:

We would check the presence of '-' in the domain part of URL. The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name so that users feel that they are dealing with a legitimate webpage.

Hence, If the URL has '-' symbol in the domain part of the URL, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [182]:
def checkPrefSuff(url):
    if "-" in urlparse(url).netloc:
        return 1
    else:
        return 0

# 3. Domain based Features:

The extracted features are categorized into;

1. DNS Record
2. Website Traffic
3. Age of Domain
3. End Period of Domain